In [1]:
import os, sys, uproot
import numpy as np
import awkward
from tqdm import tqdm
import pandas as pd

In [10]:
work_dir='/grid_mnt/vol__vol_U__u/llr/cms/hakimi/WZ_analysis'
data_dir='/data_CMS/cms/hakimi/WZ_analysis/samples/WLLJJ_WToLNu_EWK_TuneCP5_13TeV_madgraph-madspin-pythia8/RunIIAutumn18NanoAODv6-Nano25Oct2019_102X_upgrade2018_realistic_v20-v1/NANOAODSIM/'
files=os.listdir(data_dir)
input_files=[f for f in files if ".root" in f]

In [3]:
def extract_scalar_data(events, branches, entrystop=None, progressbar=False):
    data = {}

    data["event"] = events.array("event", entrystop=entrystop)

    for br in tqdm(branches, disable=not progressbar):
        data[br] = events.array(br, entrystop=entrystop).flatten()

    return pd.DataFrame(data)

In [4]:
def extract_vector_data(events, branches, entrystop=10, progressbar=False):
    def get(branch, flat=True):
        a = events.array(branch, entrystop=entrystop)
        if flat:
            return a.flatten()
        else:
            return a

    if len(branches) == 0:
        return {}

    first_branch_jagged = get(branches[0], flat=False)
    first_branch_flat = first_branch_jagged.flatten()

    event_jagged = get("event") + awkward.JaggedArray(
        first_branch_jagged.starts, first_branch_jagged.stops, np.zeros(len(first_branch_flat), dtype=np.int)
    )

    data = {}

    data["event"] = event_jagged.flatten()

    data[branches[0]] = first_branch_flat

    for br in tqdm(branches, disable=not progressbar):
        if br == branches[0]:
            continue

        if br in events:
            data[br] = get(br)
        else:
            print('Warning! Branch "' + br + '" not found in input file and skipped.')

    return pd.DataFrame(data)


In [5]:
def nanoaod_to_parquet(input_files, out_dir, entrystop=None, input_prefix="", progressbar=False):
    def save_parquet(df, name):
        df.to_parquet(os.path.join(out_dir, name + ".parquet.gzip"), compression="gzip", index=False)

    out_dir = os.path.expanduser(out_dir)

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    n_input_files = len(input_files)

    for i, input_file in enumerate(input_files):

        log_prefix = ""
        if i > 0:
            log_prefix = f"[{i+1}/{n_input_files}] "

        def log(s):
            print(log_prefix + s)

        log("Opening input file " + input_file)
        root_file = uproot.open(input_prefix + input_file)

        events = root_file["Events"]

        branches = [br.decode("ascii") for br in events.keys()]

        n_events = len(events[branches[0]])

        prefixes = list(set([b.split("_")[0] for b in branches]))

        vector_groups_present = [p for p in prefixes if "n" + p in events]

        scalar_branches = ["n" + s for s in vector_groups_present]

        for br in branches:
            if br == "event":
                continue
            if not br.split("_")[0] in vector_groups_present:
                scalar_branches.append(br)

        basename = os.path.basename(input_file)[:-5]

        log("Loading DataFrame for scalar branches")
        df_scalar = extract_scalar_data(events, scalar_branches, entrystop=entrystop, progressbar=progressbar)
        log("Saving DataFrame parquet Scalar")
        save_parquet(df_scalar, basename + "_Scalar")
        del df_scalar

        processed_branches = scalar_branches + ["event"]

        for group in vector_groups_present:
            log("Loading DataFrame for object group " + group)
            filtered_branches = list(filter(lambda br: br == group or br.startswith(group + "_"), branches))
            df = extract_vector_data(events, filtered_branches, entrystop=entrystop, progressbar=progressbar)
            log("Saving DataFrame parquet " + group)
            save_parquet(df, basename + "_" + group)
            processed_branches += filtered_branches
            del df

        # make sure we considered all the branches
        assert [b for b in branches if not b in processed_branches] == []



In [6]:
filename=data_dir+'/'+input_files[0]
root_file=uproot.open(filename)

In [7]:
events=root_file['Events']

branches=[br.decode('utf8') for br in events.keys()]

n_events=len(events[branches[0]])
print(n_events)

prefixes = list(set([b.split("_")[0] for b in branches]))

vector_groups_present = [p for p in prefixes if "n" + p in events]

scalar_branches = ["n" + s for s in vector_groups_present]

260400


In [8]:
out_dir=data_dir+'/parquet'
input_prefix=data_dir

In [11]:
nanoaod_to_parquet(input_files, out_dir, entrystop=None, input_prefix=data_dir, progressbar=True)

Opening input file ACD761A6-DBF4-2A49-A80D-E34FE198A9C4.root


  1%|          | 8/1144 [00:00<00:15, 73.86it/s]

Loading DataFrame for scalar branches


100%|██████████| 1144/1144 [00:11<00:00, 98.56it/s] 


Saving DataFrame parquet Scalar
Loading DataFrame for object group PSWeight


...

[3/3] Saving DataFrame parquet FsrPhoton
[3/3] Loading DataFrame for object group OtherPV


100%|██████████| 1/1 [00:00<00:00, 4369.07it/s]


[3/3] Saving DataFrame parquet OtherPV
[3/3] Loading DataFrame for object group SubGenJetAK8


100%|██████████| 4/4 [00:01<00:00,  3.04it/s]


[3/3] Saving DataFrame parquet SubGenJetAK8
